In [1]:
import pandas as pd

In [2]:
def remove_duplicates(lst):
    return list(dict.fromkeys(lst).keys())

In [5]:
# submisstion 8
df = pd.read_csv('submission_8.csv')
df

,id,categories,osd
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[37],1
1,49254842-f712-4875-ad09-57ebf8508bd1,219 218 52,1
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[52],1
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[160],0
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,51 52 283,0
...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[1],1
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[88],1
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[52],1
10742,5b9257ee-ee54-4936-a192-12facfa04be7,9 119 88 160 51,0


In [14]:
# submission 7
df = pd.read_csv('submission_7.csv')
df

,id,categories,osd
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[52],1
1,49254842-f712-4875-ad09-57ebf8508bd1,219 52 218 242,1
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[52],1
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[160],0
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,[51],0
...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[52],1
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[120],1
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[205],1
10742,5b9257ee-ee54-4936-a192-12facfa04be7,119 52 160 9,0


In [38]:
def select_top(string):
    if '[' not in string:
        top = [int(x) for x in string.split(' ')][0]
        return f'[{top}]'
    else:
        return string

In [41]:
df['categories'] = df['categories'].apply(select_top)

In [42]:
df

,id,categories,osd
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[52],1
1,49254842-f712-4875-ad09-57ebf8508bd1,[219],1
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[52],1
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[160],0
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,[51],0
...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[52],1
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[120],1
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[205],1
10742,5b9257ee-ee54-4936-a192-12facfa04be7,[119],0


In [43]:
df[['id', 'categories', 'osd']].to_csv('submission_11.csv', index=False)

In [5]:
df.categories = df.categories.str.split(' ')
df.categories = df.categories.apply(remove_duplicates)
df

,id,categories,osd
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[160],0.9
1,49254842-f712-4875-ad09-57ebf8508bd1,"[219, 52, 218, 242]",0.5
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[160],0.9
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[160],0.1
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,[51],0.1
...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[160],0.9
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[120],0.5
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[205],0.5
10742,5b9257ee-ee54-4936-a192-12facfa04be7,"[119, 52, 160, 9]",0.1


In [11]:
def convert_cats(lst):
    if len(lst) > 1:
        lst = ' '.join([x for x in lst])
    else:
        lst = '[' + str(lst[0]) + ']'
    return lst

In [13]:
convert_cats(['1', '2']), convert_cats(['1'])

('1 2', '[1]')

In [14]:
df.categories = df.categories.apply(convert_cats)

In [15]:
df

,id,categories,osd
0,9cdaaebd-acd7-403c-829c-af5ba519c8a7,[160],0.9
1,49254842-f712-4875-ad09-57ebf8508bd1,219 52 218 242,0.5
2,c7f61e23-a0c1-4687-ad8a-b65a1db37fdb,[160],0.9
3,67d55379-18ca-40ec-b9da-6aa7117e4e1a,[160],0.1
4,e6fdae2f-86ee-46c7-8c75-fba6a35b8f0d,[51],0.1
...,...,...,...
10739,90508480-7df0-450f-bb1e-80462661cd7b,[160],0.9
10740,01230b6e-f53c-431a-850c-b1cbe877f08b,[120],0.5
10741,6af67476-63ea-4ac1-afc3-b79b3ef019bf,[205],0.5
10742,5b9257ee-ee54-4936-a192-12facfa04be7,119 52 160 9,0.1


In [16]:
df[['id', 'categories', 'osd']].to_csv('submission_7.csv', index=False)